# Flight Planning Domain 

This notebook aims to make a short and interactive example of the Flight Planning Domain. You can find more information about it in the README file.

Concerning the python kernel to use for this notebook:
- If running locally, be sure to use an environment with scikit-decide[all] and minizinc.
- If running on colab, the next cell does it for you.
- If running on binder, the environment should be ready.

In [ ]:
# On Colab: install the library
on_colab = "google.colab" in str(get_ipython())
if on_colab:
    import glob
    import importlib
    import json
    import os
    import sys

    using_nightly_version = True

    if using_nightly_version:
        # look for nightly build download url
        release_curl_res = !curl -L   -H "Accept: application/vnd.github+json" -H "X-GitHub-Api-Version: 2022-11-28" https://api.github.com/repos/airbus/scikit-decide/releases/tags/nightly
        release_dict = json.loads(release_curl_res.s)
        release_download_url = sorted(
            release_dict["assets"], key=lambda d: d["updated_at"]
        )[-1]["browser_download_url"]
        print(release_download_url)

        # download and unzip
        !wget --output-document=release.zip {release_download_url}
        !unzip -o release.zip

        # get proper wheel name according to python version used
        wheel_pythonversion_tag = f"cp{sys.version_info.major}{sys.version_info.minor}"
        wheel_path = glob.glob(
            f"dist/scikit_decide*{wheel_pythonversion_tag}*manylinux*.whl"
        )[0]

        skdecide_pip_spec = f"{wheel_path}[all]"
    else:
        skdecide_pip_spec = "scikit-decide[all]"

    # uninstall google protobuf conflicting with ray and sb3
    ! pip uninstall -y protobuf

    # install scikit-decide with all extras
    !pip install {skdecide_pip_spec}

    # be sure to load the proper cffi (downgraded compared to the one initially on colab)
    import cffi

    importlib.reload(cffi)

    # install and configure minizinc
    !curl -o minizinc.AppImage -L https://github.com/MiniZinc/MiniZincIDE/releases/download/2.6.3/MiniZincIDE-2.6.3-x86_64.AppImage
    !chmod +x minizinc.AppImage
    !./minizinc.AppImage --appimage-extract
    os.environ["PATH"] = f"{os.getcwd()}/squashfs-root/usr/bin/:{os.environ['PATH']}"
    os.environ[
        "LD_LIBRARY_PATH"
    ] = f"{os.getcwd()}/squashfs-root/usr/lib/:{os.environ['LD_LIBRARY_PATH']}"

In [ ]:
from skdecide.hub.domain.flight_planning.domain import FlightPlanningDomain, WeatherDate
from skdecide.hub.domain.flight_planning.weather_interpolator.weather_tools.get_weather_noaa import (
    get_weather_matrix,
)
from skdecide.hub.domain.flight_planning.weather_interpolator.weather_tools.interpolator.GenericInterpolator import (
    GenericWindInterpolator,
)

### Definition of the problem

Here we will make a short haul flight from Paris Charles de Gaulle airport (ICAO : LFPG) to Toulouse-Blagnac airport (ICAO: LFBO), using an airbus A320 aircraft.

In [ ]:
origin = "LFPG"
destination = "LFBO"
aircraft = "A320"

Now, we are going to define a date, in the past 6 months, to get access to weather interpolation.

In [ ]:
weather_date = WeatherDate(day=21, month=4, year=2023)
w_dict = weather_date.to_dict()
mat = get_weather_matrix(
    year=w_dict["year"],
    month=w_dict["month"],
    day=w_dict["day"],
    forecast=w_dict["forecast"],
    delete_npz_from_local=False,
    delete_grib_from_local=False,
)
wind_interpolator = GenericWindInterpolator(file_npz=mat)

We can now define heuristic & cost function, to feed the A* solver. This aims to guide him along the airways graph to solve the problem, and get as close as possible to the optimal solution. 

The heuristic function can be either fuel, time, distance, lazy_fuel, lazy_time and None. If None, the A* will behave as a Dijkstra like search algorithm, as we give a 0 value to the A* algorithm. 

The cost function can be either fuel, time and distance. It will define the cost of the flight plan, computed during the state to state flight simulation. 

In [ ]:
heuristic = "lazy_fuel"
cost_function = "fuel"

### Definition of the Domain and Optional features 

There is many other optionnal features, as described in the README. We are now going to define the domain. 

It will initialize the Flight Planning Domain with the given features, and can take some time, especially if it needs to download the differents weather files, and if you ask for a fuel loop. 


In [ ]:
domain_factory = lambda: FlightPlanningDomain(
    origin,
    destination,
    aircraft,
    weather_date=weather_date,
    wind_interpolator=wind_interpolator,
    heuristic_name=heuristic,
    objective=cost_function,
    fuel_loop=False,
    graph_width="normal",
)
domain = domain_factory()

Then we can solve the problem : (can take time depending on )

In [ ]:
domain.solve(domain_factory, make_img=True)